In [1]:
# Don't use pls

# import requests

# url = "https://publicapi.traffy.in.th/dump-csv-chadchart/bangkok_traffy.csv"
# response = requests.get(url)

# if response.status_code == 200:
#     with open("bangkok_traffy.csv", "wb") as f:
#         f.write(response.content)
#     print("Data downloaded successfully.")
# else:
#     print(f"Failed to download data. Status code: {response.status_code}")

In [2]:
!pip install pythainlp -q
!pip install --no-cache-dir transformers sentencepiece -q

In [3]:
!gdown 1uNAYBPerV6IaiWXLPF4Efkxb4Gg-U4Cz

Downloading...
From: https://drive.google.com/uc?id=1uNAYBPerV6IaiWXLPF4Efkxb4Gg-U4Cz
To: /content/bangkok_traffy.csv
100% 362M/362M [00:01<00:00, 199MB/s]


In [4]:
import numpy as np
import pandas as pd

# Load Data

In [5]:
df = pd.read_csv("bangkok_traffy.csv")

In [6]:
# df.head()

# Select and Clean feature & target columns

In [7]:
df = df[["type", "comment", "province"]]

In [8]:
df = df[df['province'].str.contains('กรุงเทพ|bangkok',na=False,case=False)]
df = df.drop(["province"], axis=1)

In [9]:
df['type'].dropna(axis=0, inplace=True)
df['comment'].dropna(axis=0, inplace=True)

In [10]:
df['type'].fillna('', inplace=True) 
df['type'] = df['type'].astype(str) 
df['type'] = df['type'].str.strip('{}').apply(lambda x :[e for e in x.split(',') if e])

In [11]:
df = df.reset_index(drop=True)

In [12]:
df

,type,comment
0,[],ไม่มีภาพ
1,[ความสะอาด],ขยะเยอะ
2,[สายไฟ],1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
3,"[ถนน, สะพาน]",1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
4,"[น้ำท่วม, ร้องเรียน]",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
...,...,...
273112,[ความสะอาด],ไม่มีเจ้าหน้าที่มาเก็บขยะหลายวันแล้วคะ
273113,"[ถนน, คลอง]",* ปัญหา: ประชาชนพบจระเข้ 2 ตัว ขอให้ดำเนินการจ...
273114,[ถนน],* ปัญหา: ประชาชนต้องการให้เจ้าหน้าที่มาเก็บขยะ...
273115,[],ผมขับมอเตอร์ไซค์ผ่าน บริเวณสายสอง มีกลุ่มควันเ...


Preprocess: type

In [13]:
df["type"] = df["type"].apply(lambda x: np.nan if x == [] else x)
df = df.dropna().reset_index(drop=True)
df

,type,comment
0,[ความสะอาด],ขยะเยอะ
1,[สายไฟ],1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
2,"[ถนน, สะพาน]",1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
3,"[น้ำท่วม, ร้องเรียน]",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
4,"[ถนน, ร้องเรียน, น้ำท่วม, ความปลอดภัย]",1) พื้นที่นี้ขาดหน่วยงานรับผิดชอบ เนื่องจากไม่...
...,...,...
208994,"[ถนน, ท่อระบายน้ำ]",มีฝาท่อถนนเพชรบุรี ตั้งแต่ก่อนถึงทางรถไฟ ไปจนถ...
208995,[ความสะอาด],ไม่มีเจ้าหน้าที่มาเก็บขยะหลายวันแล้วคะ
208996,"[ถนน, คลอง]",* ปัญหา: ประชาชนพบจระเข้ 2 ตัว ขอให้ดำเนินการจ...
208997,[ถนน],* ปัญหา: ประชาชนต้องการให้เจ้าหน้าที่มาเก็บขยะ...


In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
transformed_data = mlb.fit_transform(df['type'])

dummy_df = pd.DataFrame(transformed_data, columns=[f'type_{label}' for label in mlb.classes_]).reset_index(drop=True)

In [44]:
dummy_df.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['type_PM2.5', 'type_การเดินทาง', 'type_กีดขวาง', 'type_คนจรจัด',
       'type_คลอง', 'type_ความปลอดภัย', 'type_ความสะอาด', 'type_จราจร',
       'type_ต้นไม้', 'type_ถนน', 'type_ทางเท้า', 'type_ท่อระบายน้ำ',
       'type_น้ำท่วม', 'type_ป้าย', 'type_ป้ายจราจร', 'type_ร้องเรียน',
       'type_สอบถาม', 'type_สะพาน', 'type_สัตว์จรจัด', 'type_สายไฟ',
       'type_ห้องน้ำ', 'type_เสนอแนะ', 'type_เสียงรบกวน', 'type_แสงสว่าง'],
      dtype='object')

In [46]:
dummy_df.sum(axis=0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


type_PM2.5           1180
type_การเดินทาง       548
type_กีดขวาง        15699
type_คนจรจัด          991
type_คลอง            8762
type_ความปลอดภัย    20869
type_ความสะอาด      20139
type_จราจร          12252
type_ต้นไม้          8171
type_ถนน            77609
type_ทางเท้า        31816
type_ท่อระบายน้ำ    14129
type_น้ำท่วม        19197
type_ป้าย            6609
type_ป้ายจราจร        257
type_ร้องเรียน       7717
type_สอบถาม          1171
type_สะพาน          12276
type_สัตว์จรจัด      4251
type_สายไฟ           9720
type_ห้องน้ำ          276
type_เสนอแนะ         1002
type_เสียงรบกวน      8899
type_แสงสว่าง       22885
dtype: int64

In [16]:
df = pd.concat([df.drop(["type"], axis=1), dummy_df], axis=1)

In [17]:
type_columns = [col for col in df.columns if col.startswith('type')]
df[type_columns] = df[type_columns].astype(int)

In [18]:
len(type_columns)

24

In [19]:
df.head()

,comment,type_PM2.5,type_การเดินทาง,type_กีดขวาง,type_คนจรจัด,type_คลอง,type_ความปลอดภัย,type_ความสะอาด,type_จราจร,type_ต้นไม้,...,type_ป้ายจราจร,type_ร้องเรียน,type_สอบถาม,type_สะพาน,type_สัตว์จรจัด,type_สายไฟ,type_ห้องน้ำ,type_เสนอแนะ,type_เสียงรบกวน,type_แสงสว่าง
0,ขยะเยอะ,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,1) พื้นที่นี้ขาดหน่วยงานรับผิดชอบ เนื่องจากไม่...,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


# Preprocess: comment

In [20]:
df['comment'].fillna('', inplace=True) 
df['comment'] = df['comment'].astype(str) 

In [21]:
df = df.sample(1_000, random_state=123)

In [22]:
import re
import string
import pythainlp
from tqdm.auto import tqdm
tqdm.pandas()

def preprocessing_comment(comment):
    # Normalize text
    try:
      comment = pythainlp.util.normalize(comment)
    except:
      print(comment)

    # Tokenize text
    tokens = pythainlp.word_tokenize(comment)

    # Remove stop words
    stopwords = pythainlp.corpus.common.thai_stopwords()
    tokens = [token for token in tokens if token not in stopwords]

    # Join tokens back into a string
    processed_comment = ' '.join(tokens)
  
    return comment

df['comment'] = df['comment'].progress_apply(preprocessing_comment)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [23]:
# df = df.sample(30_000, random_state=123)

# Train Val Test split

In [24]:
from sklearn.model_selection import train_test_split

# df, df_tmp = train_test_split(df, test_size=0.5, random_state=123)

df_train, df_rem = train_test_split(df, test_size=0.3, random_state=123)

df_val, df_test = train_test_split(df_rem, test_size=0.5, random_state=123)

# Preprocess torch

In [34]:
import torch
from torch.utils.data import Dataset

class MultiLabelTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.feature = self.data["comment"]
        self.labels = self.data.drop(["comment"], axis=1).to_numpy()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        comment = self.data['comment'].iloc[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        # Tokenize the comment
        tokenized_comment = self.tokenizer(comment, padding='max_length', truncation=True, max_length=256, return_tensors='pt')

        return {
            'input_ids': tokenized_comment['input_ids'].squeeze(),
            'attention_mask': tokenized_comment['attention_mask'].squeeze(),
            'labels': label
        }


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Step 2: Prepare the data
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", use_fast=False)
# Process your data and convert it to input features (input_ids, attention_mask, labels_binary)
train_dataset = MultiLabelTextDataset(df_train, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataset = MultiLabelTextDataset(df_val, tokenizer)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_dataset = MultiLabelTextDataset(df_test, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
# train_dataset[6]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
# next(iter(train_loader))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Model

In [38]:
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
class Classifier(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(Classifier, self).__init__()
        self.fc2 = nn.Linear(n_inputs, n_outputs)
        # self.dropout = nn.Dropout(0.15)

    def forward(self, x):
        # x = self.dropout(x)
        x = self.fc2(x)
        return x

wangchanberta_model = AutoModelForSequenceClassification.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", num_labels=512).to(device)
class Model(nn.Module):
    def __init__(self, classifier):
        super().__init__()
        self.embeding = wangchanberta_model
        # self.embeding = nn.EmbeddingBag(200_000, 512,padding_idx=1)
        # for param in self.embeding.parameters():
        #     param.requires_grad = False
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 512)
        self.dropout = nn.Dropout(0.1)
        self.classifier = classifier


    def update_freeze_embed(self, is_freeze):
        if is_freeze:
          for param in self.embeding.parameters():
              param.requires_grad = False
        else:
          for param in self.embeding.parameters():
              param.requires_grad = True
        
    def forward(self,input_ids, attention_mask):
        x = self.embeding(input_ids=input_ids, attention_mask=attention_mask).logits
        # x = self.embeding(input_ids)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        out = self.classifier(x)
        return out

classifier = Classifier(512, len(type_columns))
model = Model(classifier).to(device)
# model.update_freeze_embed(False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:

In [39]:
# # Step 3: Load the pre-trained WangchanBERTa model
# num_classes = len(type_columns)
# model = AutoModelForSequenceClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=num_classes)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# Step 4: Setup the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Model(
  (embeding): CamembertForSequenceClassification(
    (roberta): CamembertModel(
      (embeddings): CamembertEmbeddings(
        (word_embeddings): Embedding(25005, 768, padding_idx=1)
        (position_embeddings): Embedding(512, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): CamembertEncoder(
        (layer): ModuleList(
          (0-11): 12 x CamembertLayer(
            (attention): CamembertAttention(
              (self): CamembertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): CamembertSelfOutput(
              

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings('always')

optimizer = torch.optim.Adam(model.parameters(), lr=4e-5)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
criterion = torch.nn.BCEWithLogitsLoss()
scheduler = optim.lr_scheduler.StepLR(
    optimizer, 
    step_size=4, 
    gamma=0.7
    ) 
num_epochs = 16

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_correct = 0
    num_train_samples = 0

    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels_binary = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        sigmoid_outputs = torch.sigmoid(outputs)

        loss = criterion(sigmoid_outputs, labels_binary)
        loss.backward()
        optimizer.step()

        predicted_labels = (sigmoid_outputs > 0.5).float()
        train_correct += (predicted_labels == labels_binary).sum().item()

        train_loss += loss.item() * input_ids.size(0)
        num_train_samples += input_ids.size(0)

    train_loss /= num_train_samples
    train_accuracy = train_correct / num_train_samples
    train_precision = precision_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')
    train_recall = recall_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')
    train_f1 = f1_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')

    model.eval()
    valid_loss = 0
    valid_correct = 0
    num_valid_samples = 0

    with torch.no_grad():
        for batch in tqdm(valid_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels_binary = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            sigmoid_outputs = torch.sigmoid(outputs)

            loss = criterion(sigmoid_outputs, labels_binary)
            predicted_labels = (sigmoid_outputs > 0.5).float()
            valid_correct += (predicted_labels == labels_binary).sum().item()

            valid_loss += loss.item() * input_ids.size(0)
            num_valid_samples += input_ids.size(0)

    valid_loss /= num_valid_samples
    valid_accuracy = valid_correct / num_valid_samples
    valid_precision = precision_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')
    valid_recall = recall_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')
    valid_f1 = f1_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')

    print(f"Epoch: {epoch + 1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}")
    print(f"Valid Loss: {valid_loss:.4f}, Accuracy: {valid_accuracy:.4f}, Precision: {valid_precision:.4f}, Recall: {valid_recall:.4f}, F1: {valid_f1:.4f}")
    print(f"lr: {scheduler.optimizer.param_groups[0]['lr']:.8f}")
    scheduler.step()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1/16:
Train Loss: 0.8884, Accuracy: 22.2843, Precision: 0.3636, Recall: 0.1026, F1: 0.1600
Valid Loss: 0.8192, Accuracy: 22.5533, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
lr: 0.00004000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 5/16:
Train Loss: 0.6934, Accuracy: 22.5300, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Valid Loss: 0.6933, Accuracy: 22.5533, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
lr: 0.00002800


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/22 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# warnings.filterwarnings('always')

# optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
# criterion = torch.nn.BCEWithLogitsLoss()

# num_epochs = 5

# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0
#     train_correct = 0
#     num_train_samples = 0

#     for batch in tqdm(train_loader):
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels_binary = batch['labels'].to(device)

#         optimizer.zero_grad()

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
#         sigmoid_outputs = torch.sigmoid(outputs)

#         loss = criterion(sigmoid_outputs, labels_binary.float())
#         loss.backward()
#         optimizer.step()

#         predicted_labels = (sigmoid_outputs > 0.5).float()
#         train_correct += (predicted_labels == labels_binary).sum().item()

#         train_loss += loss.item() * input_ids.size(0)
#         num_train_samples += input_ids.size(0)

#     train_loss /= num_train_samples
#     train_accuracy = train_correct / num_train_samples
#     train_precision = precision_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')
#     train_recall = recall_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')
#     train_f1 = f1_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')

#     model.eval()
#     valid_loss = 0
#     valid_correct = 0
#     num_valid_samples = 0

#     with torch.no_grad():
#         for batch in tqdm(valid_loader):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels_binary = batch['labels'].to(device)

#             outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
#             sigmoid_outputs = torch.sigmoid(outputs)

#             loss = criterion(sigmoid_outputs, labels_binary.float())
#             predicted_labels = (sigmoid_outputs > 0.5).float()
#             valid_correct += (predicted_labels == labels_binary).sum().item()

#             valid_loss += loss.item() * input_ids.size(0)
#             num_valid_samples += input_ids.size(0)

#     valid_loss /= num_valid_samples
#     valid_accuracy = valid_correct / num_valid_samples
#     valid_precision = precision_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')
#     valid_recall = recall_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')
#     valid_f1 = f1_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')

#     print(f"Epoch {epoch + 1}/{num_epochs}:")
#     print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}")
#     print(f"Valid Loss: {valid_loss:.4f}, Accuracy: {valid_accuracy:.4f}, Precision: {valid_precision:.4f}, Recall: {valid_recall:.4f}, F1: {valid_f1:.4f}")

In [ ]:
model.eval()
valid_loss = 0
valid_correct = 0
num_valid_samples = 0

with torch.no_grad():
    for batch in tqdm(valid_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels_binary = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        sigmoid_outputs = torch.sigmoid(outputs)

        loss = criterion(sigmoid_outputs, labels_binary.float())
        predicted_labels = (sigmoid_outputs > 0.5).float()
        valid_correct += (predicted_labels == labels_binary).sum().item()

        valid_loss += loss.item() * input_ids.size(0)
        num_valid_samples += input_ids.size(0)
        break

print(predicted_labels)
print()
print(labels_binary)
valid_loss /= num_valid_samples
valid_accuracy = valid_correct / num_valid_samples
valid_precision = precision_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')
valid_recall = recall_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')
valid_f1 = f1_score(labels_binary.cpu().numpy(), predicted_labels.cpu().numpy(), average='micro')